In [20]:
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.perception import Concatenate
from pydrake.systems.framework import DiagramBuilder

from manipulation.scenarios import AddRgbdSensors
from manipulation.utils import ConfigureParser
from pydrake.all import MeshcatVisualizer

import sponana.utils

In [23]:
def BananaSystem():
    builder = DiagramBuilder()

    # Create the physics engine + scene graph.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)
    ConfigureParser(parser)
    parser.AddModels(
        "banana.dmd.yaml"
    )
    plant.Finalize()

    # Add a visualizer just to help us see the object.
    use_meshcat = False
    if use_meshcat:
        meshcat = builder.AddSystem(MeshcatVisualizer(scene_graph))
        builder.Connect(
            scene_graph.get_query_output_port(),
            meshcat.get_geometry_query_input_port(),
        )

    AddRgbdSensors(builder, plant, scene_graph)

    diagram = builder.Build()
    diagram.set_name("depth_camera_demo_system")
    return diagram

BananaSystem()

In [24]:
from pydrake.all import Concatenate, StartMeshcat

In [25]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7006


In [26]:
def point_cloud_processing_example():
    system = BananaSystem()

    plant = system.GetSubsystemByName("plant")

    # Evaluate the camera output ports to get the images.
    context = system.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    meshcat.Delete()
    meshcat.SetProperty("/Background", "visible", False)

    pcd = []
    for i in range(3):
        cloud = system.GetOutputPort(f"camera{i}_point_cloud").Eval(context)
        meshcat.SetObject(f"pointcloud{i}", cloud, point_size=0.001)
        meshcat.SetProperty(f"pointcloud{i}", "visible", False)

        # Crop to region of interest.
        pcd.append(
            cloud.Crop(lower_xyz=[-0.3, -0.3, -0.3], upper_xyz=[0.3, 0.3, 0.3])
        )
        meshcat.SetObject(f"pointcloud{i}_cropped", pcd[i], point_size=0.001)
        meshcat.SetProperty(f"pointcloud{i}_cropped", "visible", False)

        pcd[i].EstimateNormals(radius=0.1, num_closest=30)

        camera = plant.GetModelInstanceByName(f"camera{i}")
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd[i].FlipNormalsTowardPoint(X_C.translation())

    # Merge point clouds.  (Note: You might need something more clever here for
    # noisier point clouds; but this can often work!)
    merged_pcd = Concatenate(pcd)
    meshcat.SetObject("merged", merged_pcd, point_size=0.001)

    # Voxelize down-sample.  (Note that the normals still look reasonable)
    down_sampled_pcd = merged_pcd.VoxelizedDownSample(voxel_size=0.005)
    meshcat.SetObject("down_sampled", down_sampled_pcd, point_size=0.001)
    meshcat.SetLineSegments(
        "down_sampled_normals",
        down_sampled_pcd.xyzs(),
        down_sampled_pcd.xyzs() + 0.01 * down_sampled_pcd.normals(),
    )


point_cloud_processing_example()